In [33]:
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing

In [34]:
from collections import defaultdict
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import time

def testing_grounds(X_train2, y_train2, X_test2, y_test2, model_options, model_params):
    
    """
    Function that takes in multiple parawwmeters for cleaning type, vectorizer, model, and hyperparameter tuning and tests all possible combinations.
    
   
    
    :param model_options: a dictionary of models to test.
    :Example: 
    {'logistic_regression':LogisticRegression(), 'multinomial_nb':MultinomialNB()}
    
    :param model_params: a dictionary of model parameters to test.
    :Example: {'model_name':{ 'param_name':[param options] }}
    
    
    :return: dictionary 2 levels deep with all passed options as keys and the best parameters and scores for each combination.
    :Example: best_runs[model_options]['score','params']
    """
    num_grids = len(model_options)
    print("Total Number of Gridsearches:", num_grids)
    index=1
    best_runs = defaultdict(dict)
    # [stem/lem/none] [logistic/multinomial] [count/tfidf] [score/params]



   # X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state = 42)


    #testing the models
    for model_k, model_v in model_options.items():


        parameters = {}

        for params_k, params_v in model_params.items():
            if params_k == model_k:
                for param_k, param_v in params_v.items():
                    parameters['model'] = param_v #model__'+param_k


        pipeline = Pipeline([
            ('model', model_v)
        ])

        grid_search = GridSearchCV(pipeline, parameters, verbose=1, n_jobs=3)

        print(f"Performing grid search #{index} of {num_grids}...")
        index+=1
        print(f"Pipeline: {model_k}\n")
        print("Parameters:")
        display(parameters)
        t0 = time.time()
        grid_search.fit(X_train2, y_train2)
        print("Done in %0.3fs" % (time.time() - t0))
        print()

        print("Best train score: %0.3f" % grid_search.best_score_)
        print("Best test score: %0.3f" % grid_search.score(X_test2, y_test2))
        print("Best parameters set:")
        best_parameters = grid_search.best_estimator_.get_params()
        best_runs[model_k]['score'] = grid_search.best_score_
        best_runs[model_k]['params'] = best_parameters
        best_runs[model_k]['model'] = grid_search.best_estimator_
#         for param_name in sorted(parameters.keys()):
#             print("\t%s: %r" % (param_name, best_parameters[param_name]))
#             best_runs[model_k]['params'] = best_parameters
            
        print('\n\n')
    
    return best_runs

In [35]:
import sys
def best_model(best_runs):
    best_score = -sys.maxsize + 10
    best_params = None
    best_model = None
    for model_k, model_v in best_runs.items():


        if model_v['score'] > best_score:
            best_score = model_v['score']
            best_params = model_v['params']
            best_model = model_k

    print('Best Score:      ', best_score)
    print('Best Model:      ')
    display(best_model)
    print()
    print('Best Params:     ')
    display(best_params)
    print()

In [36]:
# Load dataset 
train = pd.read_csv('../assets/train.csv')
test = pd.read_csv('../assets/test.csv')
sample = pd.read_csv('../assets/sampleSubmission.csv')
weather = pd.read_csv('../assets/weather.csv')

In [37]:
# Get labels
labels_entire = train.WnvPresent.values

In [38]:
# Not using codesum for this benchmark
weather = weather.drop('CodeSum', axis=1)

In [39]:
# Split station 1 and 2 and join horizontally
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

In [40]:
# replace some missing values and T with -1
weather = weather.replace('M', -1)
weather = weather.replace('-', -1)
weather = weather.replace('T', -1)
weather = weather.replace(' T', -1)
weather = weather.replace('  T', -1)

In [41]:
# Functions to extract month and day from dataset
# You can also use parse_dates of Pandas.
def create_month(x):
    return x.split('-')[1]

def create_day(x):
    return x.split('-')[2]

def create_year(x):
    return x.split('-')[0]

In [42]:
train['month'] = train.Date.apply(create_month)
train['day'] = train.Date.apply(create_day)
train['year'] = train.Date.apply(create_year)
test['month'] = test.Date.apply(create_month)
test['day'] = test.Date.apply(create_day)

In [43]:
# Merge with weather data
train = train.merge(weather, on='Date')
test = test.merge(weather, on='Date')

train = train.drop(['Date'], axis = 1)
test = test.drop(['Date'], axis = 1)


In [44]:
# Convert categorical data to numbers
lbl = preprocessing.LabelEncoder()
lbl.fit(list(train['Species'].values) + list(test['Species'].values))
train['Species'] = lbl.transform(train['Species'].values)
test['Species'] = lbl.transform(test['Species'].values)

In [45]:
lbl.fit(list(train['Street'].values) + list(test['Street'].values))
train['Street'] = lbl.transform(train['Street'].values)
test['Street'] = lbl.transform(test['Street'].values)

In [46]:
lbl.fit(list(train['Trap'].values) + list(test['Trap'].values))
train['Trap'] = lbl.transform(train['Trap'].values)
test['Trap'] = lbl.transform(test['Trap'].values)

In [47]:
(train != -1).any(axis=0).head()

Address    True
Species    True
Block      True
Street     True
Trap       True
dtype: bool

In [48]:
# drop columns with -1s
train = train.loc[:,(train != -1).any(axis=0)]
test = test.loc[:,(test != -1).any(axis=0)]

In [49]:
#test train split the train off of date, multiple years' input as train and test is the last year (2013?)
#How can I test the values I get back? the labels!
mask = train['year']=='2013'
X_test = train[mask]
X_train = train[~mask]


In [50]:
X_test['year'].unique()

array(['2013'], dtype=object)

In [51]:
X_train['year'].unique()

array(['2007', '2009', '2011'], dtype=object)

In [52]:
labels_train = X_train.WnvPresent.values
labels_test = X_test.WnvPresent.values

In [53]:
# drop address columns
train = train.drop(['Address', 'AddressNumberAndStreet','WnvPresent', 'NumMosquitos', 'year'], axis = 1)
X_train = X_train.drop(['Address', 'AddressNumberAndStreet','WnvPresent', 'NumMosquitos', 'year'], axis = 1)
X_test = X_test.drop(['Address', 'AddressNumberAndStreet','WnvPresent', 'NumMosquitos', 'year'], axis = 1)

test = test.drop([ 'Address', 'AddressNumberAndStreet'], axis = 1)

In [54]:
# Random Forest Classifier 
clf = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_split=2)
clf.fit(X_train, labels_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [55]:
clf.score(X_test, labels_test)

0.9000836120401338

In [56]:
labels_test.shape

(2392,)

In [57]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier#, VotingClassifier
import xgboost as xbg
from sklearn.linear_model import LogisticRegressionCV

In [58]:
objs = []
for col in X_train.columns:
    if X_train[col].dtypes == 'O':
        objs.append(col)
        
X_train[objs[9:]].head()

,Depth_x,SnowFall_x,PrecipTotal_x,StnPressure_x,SeaLevel_x,AvgSpeed_x,Tavg_y,WetBulb_y,Heat_y,Cool_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,AvgSpeed_y
0,0,0.0,0.00,29.39,30.11,6.5,77,66,0,12,0.00,29.44,30.09,7.4
1,0,0.0,0.00,29.39,30.11,6.5,77,66,0,12,0.00,29.44,30.09,7.4
2,0,0.0,0.00,29.39,30.11,6.5,77,66,0,12,0.00,29.44,30.09,7.4
3,0,0.0,0.00,29.39,30.11,6.5,77,66,0,12,0.00,29.44,30.09,7.4
4,0,0.0,0.00,29.39,30.11,6.5,77,66,0,12,0.00,29.44,30.09,7.4


In [59]:
ints = objs[0:10]
floats = objs[10:15]
ints.extend(objs[15:19])
floats.extend(objs[19:24])


In [60]:
len(floats) + len(ints) == len(objs)
#len(objs)

True

In [61]:
X_train[ints] = X_train[ints].astype(int)
X_train[floats] = X_train[floats].astype(float)

X_test[ints] = X_test[ints].astype(int)
X_test[floats] = X_test[floats].astype(float)



In [62]:
X_train.shape[1], test.shape[1]

(40, 41)

In [63]:
X_train.dtypes


Species              int64
Block                int64
Street               int64
Trap                 int64
Latitude           float64
Longitude          float64
AddressAccuracy      int64
month                int64
day                  int64
Tmax_x               int64
Tmin_x               int64
Tavg_x               int64
Depart_x             int64
DewPoint_x           int64
WetBulb_x            int64
Heat_x               int64
Cool_x               int64
Sunrise_x            int64
Sunset_x             int64
Depth_x              int64
SnowFall_x         float64
PrecipTotal_x      float64
StnPressure_x      float64
SeaLevel_x         float64
ResultSpeed_x      float64
ResultDir_x          int64
AvgSpeed_x         float64
Tmax_y               int64
Tmin_y               int64
Tavg_y               int64
DewPoint_y           int64
WetBulb_y            int64
Heat_y               int64
Cool_y               int64
PrecipTotal_y      float64
StnPressure_y      float64
SeaLevel_y         float64
R

In [32]:
# testing a bunch of models in the background while I try to feature engineer. No real hyper-parameter tuning
n_est = 1000

model_options = {
    'logistic_regression':LogisticRegressionCV(random_state=42, solver='sag', max_iter = 10000),
#     'linreg':LinearRegression(),
#     'xgboost':xbg.XGBRegressor(),
#     'random_forest_regressor':RandomForestRegressor(n_estimators=n_est),
#     'extra_trees_regressor':ExtraTreesRegressor(n_estimators=n_est),
#     'ada_boost_regressor':AdaBoostRegressor(n_estimators=n_est),
    #'gradient':GradientBoostingRegressor(),
    'random_forest_classifier':RandomForestClassifier(n_estimators=n_est),
    'extra_trees_classifier':ExtraTreesClassifier(n_estimators=n_est),
    'ada_boost_classifier':AdaBoostClassifier(n_estimators=n_est),
    'gradient_classifier':GradientBoostingClassifier()
    
}
model_params = {
    
}

best_runs = testing_grounds(X_train, labels_train, X_test, labels_test, model_options, model_params)

Total Number of Gridsearches: 5
Performing grid search #1 of 5...
Pipeline: logistic_regression

Parameters:


{}

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   42.3s finished


KeyboardInterrupt: 

In [ ]:
for k, v in best_runs.items():
    print(f'Model: {k}')
    print(f'    Score: {v["score"]}')

print('\n\n\n\n\n')
best_model(best_runs)

In [ ]:
#prior to changes in the function
for k, v in best_runs.items():
    print(f'Model: {k}')
    print(f'    Score: {v["score"]}')

print('\n\n\n\n\n')
best_model(best_runs)

In [64]:
logreg = LogisticRegressionCV(random_state=42, solver='sag', max_iter = 10000)
logreg.fit(X_train, labels_train)
logreg.score(X_test, labels_test)


0.9000836120401338

In [65]:
logrg2 = best_runs['logistic_regression']['model']
logrg2.fit(X_train, labels_train)
logrg2.score(X_test, labels_test)

NameError: name 'best_runs' is not defined

In [ ]:
logrg2.score(X_train, labels_train)

In [ ]:
pl = Pipeline([('model',LogisticRegressionCV())])

In [ ]:
pl.fit(X_train, labels)

In [66]:
# These two cells are for the test submission to kaggle

# Random Forest Classifier 
clf_final = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_split=2)
clf_final.fit(train, labels_entire)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [67]:
# create predictions and submission file
predictions = clf_final.predict_proba(test.drop(columns='Id'))[:,1]
sample['WnvPresent'] = predictions
sample.to_csv('../assets/submission_4_randomforest.csv', index=False)

In [68]:
len(predictions)

116293

In [69]:
sample.head(10)

,Id,WnvPresent
0,1,0.021683
1,2,0.006200
2,3,0.016132
3,4,0.006200
4,5,0.006067
5,6,0.006067
6,7,0.006067
7,8,0.016132
8,9,0.008300
9,10,0.005300


In [70]:
test['Id'].head(10)

0     1
1     2
2     3
3     4
4     5
5     6
6     7
7     8
8     9
9    10
Name: Id, dtype: int64

In [71]:
result_df = test.merge(sample, on='Id')

In [72]:
result_df.head(10)

,Id,Species,Block,Street,Trap,Latitude,Longitude,AddressAccuracy,month,day,...,WetBulb_y,Heat_y,Cool_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y,WnvPresent
0,1,2,41,36,1,41.954690,-87.800991,9,06,11,...,64,0,11,0.00,29.34,29.97,9.4,18,10.4,0.021683
1,2,3,41,36,1,41.954690,-87.800991,9,06,11,...,64,0,11,0.00,29.34,29.97,9.4,18,10.4,0.006200
2,3,1,41,36,1,41.954690,-87.800991,9,06,11,...,64,0,11,0.00,29.34,29.97,9.4,18,10.4,0.016132
3,4,4,41,36,1,41.954690,-87.800991,9,06,11,...,64,0,11,0.00,29.34,29.97,9.4,18,10.4,0.006200
4,5,6,41,36,1,41.954690,-87.800991,9,06,11,...,64,0,11,0.00,29.34,29.97,9.4,18,10.4,0.006067
5,6,5,41,36,1,41.954690,-87.800991,9,06,11,...,64,0,11,0.00,29.34,29.97,9.4,18,10.4,0.006067
6,7,7,41,36,1,41.954690,-87.800991,9,06,11,...,64,0,11,0.00,29.34,29.97,9.4,18,10.4,0.006067
7,8,0,41,36,1,41.954690,-87.800991,9,06,11,...,64,0,11,0.00,29.34,29.97,9.4,18,10.4,0.016132
8,9,2,62,30,8,41.994991,-87.769279,9,06,11,...,64,0,11,0.00,29.34,29.97,9.4,18,10.4,0.008300
9,10,3,62,30,8,41.994991,-87.769279,9,06,11,...,64,0,11,0.00,29.34,29.97,9.4,18,10.4,0.005300


In [73]:
minimum = result_df['WnvPresent'].min()
maximum = result_df['WnvPresent'].max()

In [74]:
midpoint = ((maximum - minimum)/2) + minimum
new_results = result_df[result_df['WnvPresent']>=midpoint]

In [75]:
new_results=new_results.sort_values('WnvPresent', ascending=False)
new_results.head()

,Id,Species,Block,Street,Trap,Latitude,Longitude,AddressAccuracy,month,day,...,WetBulb_y,Heat_y,Cool_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y,WnvPresent
108090,108091,2,60,15,7,41.990284,-87.797127,8,08,21,...,73,0,12,1.29,29.26,29.90,4.0,18,6.3,0.615017
108092,108093,1,60,15,7,41.990284,-87.797127,8,08,21,...,73,0,12,1.29,29.26,29.90,4.0,18,6.3,0.613665
108097,108098,0,60,15,7,41.990284,-87.797127,8,08,21,...,73,0,12,1.29,29.26,29.90,4.0,18,6.3,0.612665
107710,107711,2,41,36,1,41.954690,-87.800991,9,08,21,...,73,0,12,1.29,29.26,29.90,4.0,18,6.3,0.561317
107709,107710,2,41,36,1,41.954690,-87.800991,9,08,21,...,73,0,12,1.29,29.26,29.90,4.0,18,6.3,0.561317


In [76]:
result_df = result_df.sort_values('WnvPresent', ascending=False)
result_df


,Id,Species,Block,Street,Trap,Latitude,Longitude,AddressAccuracy,month,day,...,WetBulb_y,Heat_y,Cool_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y,WnvPresent
108090,108091,2,60,15,7,41.990284,-87.797127,8,08,21,...,73,0,12,1.29,29.26,29.90,4.0,18,6.3,0.615017
108092,108093,1,60,15,7,41.990284,-87.797127,8,08,21,...,73,0,12,1.29,29.26,29.90,4.0,18,6.3,0.613665
108097,108098,0,60,15,7,41.990284,-87.797127,8,08,21,...,73,0,12,1.29,29.26,29.90,4.0,18,6.3,0.612665
107709,107710,2,41,36,1,41.954690,-87.800991,9,08,21,...,73,0,12,1.29,29.26,29.90,4.0,18,6.3,0.561317
107710,107711,2,41,36,1,41.954690,-87.800991,9,08,21,...,73,0,12,1.29,29.26,29.90,4.0,18,6.3,0.561317
107873,107874,1,36,39,11,41.944869,-87.832763,8,08,21,...,73,0,12,1.29,29.26,29.90,4.0,18,6.3,0.537676
107874,107875,1,36,39,11,41.944869,-87.832763,8,08,21,...,73,0,12,1.29,29.26,29.90,4.0,18,6.3,0.537676
107879,107880,0,36,39,11,41.944869,-87.832763,8,08,21,...,73,0,12,1.29,29.26,29.90,4.0,18,6.3,0.536676
24395,24396,1,41,36,1,41.954690,-87.800991,9,09,03,...,65,0,8,0.00,29.34,29.99,11.2,1,12.1,0.533867
24400,24401,0,41,36,1,41.954690,-87.800991,9,09,03,...,65,0,8,0.00,29.34,29.99,11.2,1,12.1,0.531534


In [77]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
#%matplotlib inline

from sklearn.neighbors import KernelDensity

mapdata = np.loadtxt("../assets/mapdata_copyright_openstreetmap_contributors.txt")
traps = result_df[['Trap','Longitude', 'Latitude', 'WnvPresent']]

# these are needed for plotting densities over map image,
# it changes alpha channel?

# see,
# Meaning of the colormap._lut list in matplotlib.color
# http://stackoverflow.com/questions/18035411/meaning-of-the-colormap-lut-list-in-matplotlib-color

alpha_cm = plt.cm.Reds
alpha_cm._init()
alpha_cm._lut[:-3,-1] = abs(np.logspace(0, 1, alpha_cm.N) / 10 - 1)[::-1]


aspect = mapdata.shape[0] * 1.0 / mapdata.shape[1]
lon_lat_box = (-88, -87.5, 41.6, 42.1) # xmin, xmax, ymin, ymax

sigthings = traps[traps['WnvPresent'] > 0]
sigthings = sigthings.groupby(['Trap','Longitude', 'Latitude']).max()['WnvPresent'].reset_index()
X = sigthings[['Longitude', 'Latitude']].values
kd = KernelDensity(bandwidth=0.02)
kd.fit(X)

xv,yv = np.meshgrid(np.linspace(-88, -87.5, 100), np.linspace(41.6, 42.1, 100))
gridpoints = np.array([xv.ravel(),yv.ravel()]).T
zv = np.exp(kd.score_samples(gridpoints).reshape(100,100))
plt.figure(figsize=(10,14))
plt.imshow(mapdata, 
           cmap=plt.get_cmap('gray'), 
           extent=lon_lat_box, 
           aspect=aspect)
plt.imshow(zv, 
           origin='lower', 
           cmap=alpha_cm, 
           extent=lon_lat_box, 
           aspect=aspect)

# -> how to use 'extent' in matplotlib.pyplot.imshow
# http://stackoverflow.com/questions/6999621/how-to-use-extent-in-matplotlib-pyplot-imshow

locations = traps[['Longitude', 'Latitude']].drop_duplicates().values
plt.scatter(locations[:,0], locations[:,1], marker='x')

plt.savefig('heatmap.png')